# Minesweeper: Game Rules and Mechanics

## Objective
The goal of Minesweeper is to clear a board containing hidden mines without detonating any. To win, you must reveal all squares that do not contain mines. 

## Game Board
- The board is a grid of squares, each of which may contain a mine or be empty.
- Each square can be one of the following:
  - **Unrevealed**: Hidden; the player cannot see what lies beneath.
  - **Revealed**: Exposed; the player can see whether it contains a mine or a number.
  - **Flagged**: Marked by the player to indicate a suspected mine.

## Core Rules
1. **Revealing a Square**: 
   - When a player clicks on an unrevealed square, one of two things will happen:
     - **Mine**: If the square contains a mine, the game ends (a loss).
     - **Empty Square**: The square reveals a number, indicating how many adjacent squares (horizontally, vertically, and diagonally) contain mines.
2. **Numbered Squares**:
   - Numbers range from 1 to 8, corresponding to the count of adjacent mines.
   - If the revealed square has no adjacent mines (indicated by a blank or "0"), it reveals neighboring squares automatically, creating a chain effect until all adjacent numbers are uncovered.
3. **Flagging Mines**:
   - Players can flag squares to indicate they suspect a mine's location.
   - However, to simplify simulating the game, we will not implement flagging in this project.
     - AI agents are perfectly capable of playing without flagging.
4. **Winning the Game**:
   - To win, the player must reveal all squares that do not contain mines.
   - Once all non-mine squares are revealed, the game is won, regardless of remaining flagged squares.



In [13]:
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

class Minesweeper:
    def __init__(self, rows, cols, num_mines):
        """
        Initialize the Minesweeper game.
        """
        self.rows = rows
        self.cols = cols
        self.num_mines = num_mines
        self.board = np.zeros((rows, cols), dtype=int)
        self.internal_state = np.full((rows, cols), None)
        self.revealed = set()
        self.mines = set()
        self.last_move = None  # Initialize last_move
        self.place_mines()
        self.calculate_hints()
        self.initialize_internal_state()

    def place_mines(self):
        """
        Randomly place mines on the board.
        Mines are represented by -1.
        """
        all_positions = [(r, c) for r in range(self.rows) for c in range(self.cols)]
        self.mines = set(random.sample(all_positions, self.num_mines))
        for (r, c) in self.mines:
            self.board[r][c] = -1

    def calculate_hints(self):
        """
        For each cell that is not a mine, calculate the number of neighboring mines.
        """
        for r in range(self.rows):
            for c in range(self.cols):
                if self.board[r][c] == -1:
                    continue
                count = 0
                for dr in [-1, 0, 1]:
                    for dc in [-1, 0, 1]:
                        nr, nc = r + dr, c + dc
                        if dr == 0 and dc == 0:
                            continue
                        if 0 <= nr < self.rows and 0 <= nc < self.cols:
                            if self.board[nr][nc] == -1:
                                count += 1
                self.board[r][c] = count

    def initialize_internal_state(self):
        """
        Initialize the internal state grid.
        None represents unrevealed cells.
        """
        self.internal_state = np.full((self.rows, self.cols), None)
        self.revealed = set()
        self.last_move = None  # Reset last_move

    def reveal_square(self, r, c):
        """
        Reveal the cell at (r, c).
        Returns False if a mine is revealed, True otherwise.
        """
        if (r, c) in self.revealed:
            return True  # Already revealed

        self.last_move = (r, c)  # Record the last move

        if self.board[r][c] == -1:
            self.internal_state[r][c] = -1  # Reveal mine
            self.revealed.add((r, c))
            return False  # Mine revealed

        # Initialize a queue for iterative BFS
        queue = [(r, c)]

        while queue:
            current_r, current_c = queue.pop(0)
            
            if (current_r, current_c) in self.revealed:
                continue  # Skip already revealed cells

            # Reveal the current cell
            self.internal_state[current_r][current_c] = self.board[current_r][current_c]
            self.revealed.add((current_r, current_c))

            # If the cell has zero neighboring mines, add its neighbors to the queue
            if self.board[current_r][current_c] == 0:
                for dr in [-1, 0, 1]:
                    for dc in [-1, 0, 1]:
                        nr, nc = current_r + dr, current_c + dc
                        if dr == 0 and dc == 0:
                            continue  # Skip the current cell itself
                        if 0 <= nr < self.rows and 0 <= nc < self.cols:
                            if (nr, nc) not in self.revealed and (nr, nc) not in queue:
                                queue.append((nr, nc))

        return True


    def display_board(self):
        """
        Display the actual board (for debugging purposes).
        """
        display = self.board.copy()
        display[display == -1] = -1
        for r in range(self.rows):
            row = ''
            for c in range(self.cols):
                if display[r][c] == -1:
                    row += ' * '
                else:
                    row += f' {display[r][c]} '
            print(row)
        print()

    def plot_board(self):
        """
        Plot the current state of the board using Seaborn's heatmap.
        - Unrevealed cells are shown in a specific color (e.g., gray).
        - Revealed numbers are colored based on their value.
        - Mines are highlighted if revealed.
        - The last revealed square is highlighted with a red edge.
        """
        # Create a numerical representation for plotting
        plot_data = np.full((self.rows, self.cols), -2)  # -2 for unrevealed

        for r in range(self.rows):
            for c in range(self.cols):
                if (r, c) in self.revealed:
                    plot_data[r][c] = self.internal_state[r][c]
                else:
                    plot_data[r][c] = -2  # Unrevealed

        # Define a custom colormap
        cmap = ListedColormap([
            'lightgray',  # -2 Unrevealed
            'black',      # -1 Mine
            'white',      # 0
            'lightblue',  # 1
            'blue',       # 2
            'darkblue',   # 3
            'purple',     # 4
            'red',        # 5
            'darkred',    # 6
            'orange',     # 7
            'brown',      # 8
            'black'       # 9 (Not used, just in case)
        ])

        # Define the range and norm
        bounds = [-2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
        norm = plt.matplotlib.colors.BoundaryNorm(bounds, cmap.N)

        plt.figure(figsize=(10, 10))
        ax = sns.heatmap(plot_data, cmap=cmap, norm=norm, linewidths=0.5, linecolor='black', cbar=False)

        # Add annotations
        for r in range(self.rows):
            for c in range(self.cols):
                value = plot_data[r][c]
                if value == -2:
                    plt.text(c + 0.5, r + 0.5, '?', 
                            horizontalalignment='center',
                            verticalalignment='center',
                            fontsize=12, color='black')
                elif value > 0:
                    plt.text(c + 0.5, r + 0.5, str(value), 
                            horizontalalignment='center',
                            verticalalignment='center',
                            fontsize=12, color='white')
                elif value == 0:
                    plt.text(c + 0.5, r + 0.5, '0', 
                            horizontalalignment='center',
                            verticalalignment='center',
                            fontsize=12)

        # Highlight the last move
        if self.last_move and self.last_move in self.revealed:
            r, c = self.last_move
            # Draw a red rectangle around the last move
            ax.add_patch(plt.Rectangle((c, r), 1, 1, fill=False, edgecolor='red', linewidth=3))

        plt.title('Minesweeper Board')
        plt.axis('off')
        plt.gca().invert_yaxis()
        

    def plot_mines(self):
        """
        Plot the positions of all mines on the board using a heatmap.
        Mines are highlighted in red, and other cells are shown in white.
        """
        # Create a matrix where mines are 1 and other cells are 0
        mine_matrix = np.zeros((self.rows, self.cols))
        for (r, c) in self.mines:
            mine_matrix[r][c] = 1

        # Define a custom colormap: white for non-mines, red for mines
        cmap = ListedColormap(['grey', 'red'])
        plt.figure(figsize=(10, 10))
        # Plot the heatmap
        sns.heatmap(mine_matrix, cmap=cmap, linewidths=0.5, linecolor='black', cbar=False)


        plt.title('Mine Positions')
        plt.axis('off')
        plt.gca().invert_yaxis()
        


The task is to apply these 7 search algorithms to solve the problem

1. Breadth-first search
2. Uniform-cost search
3. Depth-first search
4. Depth-limited search
5. Iterative deepening search
6. Greedy search
7. A* search

Since the task of solving minesweeper with an agent is not suitable for the application of these 7 algorithms, we will focus first on solving the sub-problem of implementing the `reveal_square` function in Minesweeper. This function is responsible for revealing a square on the board and cascading the reveal to adjacent squares if the revealed square is empty (i.e., has no neighboring mines). The `reveal_square` function is a crucial component of the Minesweeper game logic, and implementing it correctly is essential for the game to function as expected.

### Parameters
- **`r`** (int): The row index of the square to reveal.
- **`c`** (int): The column index of the square to reveal.

### Returns
- **`False`** if a mine is revealed, indicating the game is lost.
- **`True`** if no mine is revealed, allowing the game to continue.

## Applicable Algorithms

The following search algorithms are applicable to implementing the `reveal_square` function in Minesweeper. These algorithms facilitate the efficient traversal and revelation of squares on the game board, especially when cascading reveals are necessary due to empty squares (i.e., squares with no neighboring mines).

### Breadth-First Search (BFS)

**Purpose**: BFS explores the game board level by level, starting from the initial square and moving outward to all neighboring squares before proceeding to the next level of neighbors.

**Application in `reveal_square`**:
- When a square with no neighboring mines is revealed, BFS can systematically reveal all adjacent squares.
- BFS ensures that all squares at the current distance from the initial square are revealed before moving deeper, which is efficient for uncovering large contiguous areas without mines.
- This approach prevents stack overflow issues that can occur with deep recursion in large boards, as BFS uses an iterative approach with a queue.

### Depth-First Search (DFS)

**Purpose**: DFS explores as far as possible along each branch before backtracking, delving deep into one area of the board before moving to another.

**Application in `reveal_square`**:
- DFS can be used to recursively reveal squares starting from the initial square, diving deep into adjacent empty squares.
- This method is straightforward to implement using recursion or an explicit stack, making it intuitive for handling the cascading reveal logic.
- While DFS may explore a particular path extensively before others, it effectively uncovers all connected empty squares, ensuring comprehensive revelation.

### Depth-Limited Search (DLS)

**Purpose**: DLS is a variation of DFS that imposes a limit on the depth of exploration to prevent excessive resource consumption and potential infinite loops.

**Application in `reveal_square`**:
- In scenarios where the game board is exceptionally large, DLS can limit the depth of recursive reveals to maintain performance and avoid stack overflows.
- By setting an appropriate depth limit, DLS ensures that the `reveal_square` function remains efficient, especially in edge cases with extensive empty regions.
- If the depth limit is reached without revealing all necessary squares, additional strategies (such as increasing the limit or switching to another algorithm) can be employed to complete the revelation.

### Iterative Deepening Search (IDS)

**Purpose**: IDS combines the space efficiency of DFS with the breadth-wise exploration of BFS by repeatedly applying DLS with increasing depth limits.

**Application in `reveal_square`**:
- IDS can be utilized to progressively reveal squares, starting with a shallow depth and incrementally increasing it to uncover more distant squares.
- This approach ensures that the revelation process remains both thorough and resource-conscious, adapting to the complexity of the board dynamically.
- IDS is particularly useful in situations where the optimal depth for revelation is unknown, providing a balanced method that mitigates the limitations of both BFS and DFS individually.




### Non-Applicable Algorithms
The following algorithms are not applicable to the `reveal_square` function in its current form:

1. **Uniform-Cost Search (UCS)**
   - **Purpose**: UCS is designed to find the least-cost path in a graph where each edge has an associated cost.
   - **Why Not Applicable**: The Minesweeper cell-revealing process doesn't involve varying costs between cells. Each move (revealing a cell) has an implicit, uniform cost, rendering UCS unnecessary.

2. **Greedy Search**
   - **Purpose**: Greedy Search prioritizes nodes based on a heuristic that estimates "closeness" to a goal, aiming to reach the goal as quickly as possible.
   - **Why Not Applicable**: Minesweeper’s cell-revealing process doesn’t benefit from heuristic estimations. There is no specific "goal" beyond safely revealing cells, and all cells are treated equally, without prioritization.

3. **Astar Search**
   - **Purpose**: A* combines features of UCS and Greedy Search by using both path cost and heuristic estimates to find the most efficient path.
   - **Why Not Applicable**: Like UCS and Greedy Search, A* requires path costs and heuristic functions, which are irrelevant in Minesweeper. The game’s cell-revealing process has no such metrics, making A* inapplicable.




In [14]:
def reveal_square_bfs(self, r, c):
    """
    BFS IMPLEMENTATION
    same as the one used in the above simulation of minesweeper
    """
    if (r, c) in self.revealed:
        return True  # Already revealed

    self.last_move = (r, c)  # Record the last move

    if self.board[r][c] == -1:
        self.internal_state[r][c] = -1  # Reveal mine
        self.revealed.add((r, c))
        return False  # Mine revealed

    # Initialize a queue for iterative BFS
    queue = [(r, c)]

    while queue:
        current_r, current_c = queue.pop(0)
        
        if (current_r, current_c) in self.revealed:
            continue  # Skip already revealed cells

        # Reveal the current cell
        self.internal_state[current_r][current_c] = self.board[current_r][current_c]
        self.revealed.add((current_r, current_c))

        # If the cell has zero neighboring mines, add its neighbors to the queue
        if self.board[current_r][current_c] == 0:
            for dr in [-1, 0, 1]:
                for dc in [-1, 0, 1]:
                    nr, nc = current_r + dr, current_c + dc
                    if dr == 0 and dc == 0:
                        continue  # Skip the current cell itself
                    if 0 <= nr < self.rows and 0 <= nc < self.cols:
                        if (nr, nc) not in self.revealed and (nr, nc) not in queue:
                            queue.append((nr, nc))

    return True


def reveal_square_dfs(self, r, c):
    """
    DFS IMPLEMENTATION
    Reveals squares using Depth-First Search.
    """
    # If the cell is already revealed, no action is needed
    if (r, c) in self.revealed:
        return True  # Already revealed

    # Record the last move
    self.last_move = (r, c)

    # If the selected cell is a mine, reveal it and end the game
    if self.board[r][c] == -1:
        self.internal_state[r][c] = -1  # Reveal mine
        self.revealed.add((r, c))
        return False  # Mine revealed

    # Recursive DFS function
    def dfs(current_r, current_c):
        # Base case: If out of bounds or already revealed, return
        if (current_r, current_c) in self.revealed:
            return
        if not (0 <= current_r < self.rows and 0 <= current_c < self.cols):
            return

        # Reveal the current cell
        self.internal_state[current_r][current_c] = self.board[current_r][current_c]
        self.revealed.add((current_r, current_c))

        # If the cell has zero neighboring mines, recursively reveal its neighbors
        if self.board[current_r][current_c] == 0:
            for dr in [-1, 0, 1]:
                for dc in [-1, 0, 1]:
                    nr, nc = current_r + dr, current_c + dc
                    if dr == 0 and dc == 0:
                        continue  # Skip the current cell itself
                    if 0 <= nr < self.rows and 0 <= nc < self.cols:
                        if (nr, nc) not in self.revealed:
                            dfs(nr, nc)

    # Start DFS from the initial cell
    dfs(r, c)

    return True


def reveal_square_dls(self, r, c, limit=10):
    """
    Depth-Limited Search (DLS) IMPLEMENTATION
    Reveals squares up to a specified depth from the initial square (r, c).
    """
    if (r, c) in self.revealed:
        return True  # Already revealed

    self.last_move = (r, c)  # Record the last move

    if self.board[r][c] == -1:
        self.internal_state[r][c] = -1  # Reveal mine
        self.revealed.add((r, c))
        return False  # Mine revealed

    def dls(current_r, current_c, current_depth):
        if current_depth > limit:
            return True  # Reached depth limit

        if (current_r, current_c) in self.revealed:
            return True  # Already revealed

        # Reveal the current cell
        self.internal_state[current_r][current_c] = self.board[current_r][current_c]
        self.revealed.add((current_r, current_c))

        # If the cell has zero neighboring mines, recurse on its neighbors
        if self.board[current_r][current_c] == 0:
            for dr in [-1, 0, 1]:
                for dc in [-1, 0, 1]:
                    nr, nc = current_r + dr, current_c + dc
                    if dr == 0 and dc == 0:
                        continue  # Skip the current cell itself
                    if 0 <= nr < self.rows and 0 <= nc < self.cols:
                        if (nr, nc) not in self.revealed:
                            if self.board[nr][nc] == -1:
                                self.internal_state[nr][nc] = -1  # Reveal mine
                                self.revealed.add((nr, nc))
                                return False  # Mine revealed
                            result = dls(nr, nc, current_depth + 1)
                            if not result:
                                return False  # Propagate mine found

        return True

    # Start the Depth-Limited Search from the initial square
    return dls(r, c, 0)

def reveal_square_ids(self, r, c):
    """
    Iterative Deepening Search (IDS) IMPLEMENTATION
    Reveals cells in a Minesweeper board using IDS.
    """
    if (r, c) in self.revealed:
        return True  # Already revealed

    self.last_move = (r, c)  # Record the last move

    if self.board[r][c] == -1:
        self.internal_state[r][c] = -1  # Reveal mine
        self.revealed.add((r, c))
        return False  # Mine revealed

    def dls(current_r, current_c, depth, visited):
        if depth < 0:
            return
        if (current_r, current_c) in visited:
            return
        visited.add((current_r, current_c))

        # Reveal the current cell
        self.internal_state[current_r][current_c] = self.board[current_r][current_c]
        self.revealed.add((current_r, current_c))

        # If the cell has zero neighboring mines, explore its neighbors
        if self.board[current_r][current_c] == 0 and depth > 0:
            for dr in [-1, 0, 1]:
                for dc in [-1, 0, 1]:
                    nr, nc = current_r + dr, current_c + dc
                    if dr == 0 and dc == 0:
                        continue  # Skip the current cell itself
                    if 0 <= nr < self.rows and 0 <= nc < self.cols:
                        if (nr, nc) not in self.revealed:
                            dls(nr, nc, depth - 1, visited)

    max_depth = 0
    while True:
        visited = set()
        dls(r, c, max_depth, visited)
        # If no new cells were revealed at this depth, we've finished
        if not visited:
            break
        max_depth += 1
        # Optional: Add a condition to prevent infinite loops
        if max_depth > self.rows * self.cols:
            break

    return True



In [2]:
class Agent:
    def __init__(self):
        """
        Base Agent class. To be extended by specific agent implementations.
        """
        pass

    def choose_move(self, game):
        """
        Decide the next move based on the current game state.

        Parameters:
        - game (Minesweeper): The current Minesweeper game instance.

        Returns:
        - tuple: (row, column) of the next move.
        """
        raise NotImplementedError("This method should be implemented by subclasses.")

class RandomAgent(Agent):
    def __init__(self):
        """
        Agent that chooses moves randomly from unrevealed cells.
        """
        super().__init__()

    def choose_move(self, game):
        """
        Choose a random unrevealed cell to reveal.

        Parameters:
        - game (Minesweeper): The current Minesweeper game instance.

        Returns:
        - tuple: (row, column) of the next move.
        """
        unrevealed = [(r, c) for r in range(game.rows) 
                              for c in range(game.cols) 
                              if (r, c) not in game.revealed]
        if not unrevealed:
            return None  # No moves left
        return random.choice(unrevealed)
    
    
class LogicAgent(Agent):
    def __init__(self):
        """
        Deterministic Rule-Based Agent that uses basic Minesweeper logic to choose moves.
        """
        super().__init__()
        self.suspected_mines = set()
        self.safe_moves = set()

    def choose_move(self, game):
        """
        Choose the next move based on Minesweeper logic.

        Parameters:
        - game (Minesweeper): The current Minesweeper game instance.

        Returns:
        - tuple: (row, column) of the next move, or None if no move can be determined.
        """
        self.update_knowledge(game)

        # If there are known safe moves, prioritize them
        if self.safe_moves:
            move = self.safe_moves.pop()
            return move

        # If no safe moves, pick a random move excluding suspected mines
        move = self.pick_random_move(game)
        return move

    def update_knowledge(self, game):
        """
        Update the agent's knowledge base by applying inference rules.

        Parameters:
        - game (Minesweeper): The current Minesweeper game instance.
        """
        changed = True
        while changed:
            changed = False
            for (r, c) in list(game.revealed):
                clue = game.internal_state[r][c]
                if clue is None or clue == -1:
                    continue

                # Get neighbors
                neighbors = self.get_neighbors(game, r, c)
                unrevealed = [cell for cell in neighbors if cell not in game.revealed and cell not in self.suspected_mines]

                # Count how many neighboring cells are already suspected mines
                suspected = [cell for cell in neighbors if cell in self.suspected_mines]

                # Rule 1: If number on cell equals number of suspected mines, remaining neighbors are safe
                if clue == len(suspected):
                    for cell in unrevealed:
                        if cell not in self.safe_moves:
                            self.safe_moves.add(cell)
                            changed = True

                # Rule 2: If number on cell equals suspected mines plus unrevealed neighbors, all unrevealed neighbors are mines
                elif clue == len(suspected) + len(unrevealed):
                    for cell in unrevealed:
                        if cell not in self.suspected_mines:
                            self.suspected_mines.add(cell)
                            changed = True

    def get_neighbors(self, game, r, c):
        """
        Get all valid neighboring cells around (r, c).

        Parameters:
        - game (Minesweeper): The current Minesweeper game instance.
        - r (int): Row index.
        - c (int): Column index.

        Returns:
        - list of tuples: Neighboring cell coordinates.
        """
        neighbors = []
        for dr in [-1, 0, 1]:
            for dc in [-1, 0, 1]:
                nr, nc = r + dr, c + dc
                if dr == 0 and dc == 0:
                    continue
                if 0 <= nr < game.rows and 0 <= nc < game.cols:
                    neighbors.append((nr, nc))
        return neighbors

    def pick_random_move(self, game):
        """
        Pick a random move from the remaining unrevealed cells excluding suspected mines.

        Parameters:
        - game (Minesweeper): The current Minesweeper game instance.

        Returns:
        - tuple: (row, column) of the next move, or None if no move can be determined.
        """
        unrevealed = [(r, c) for r in range(game.rows) 
                              for c in range(game.cols) 
                              if (r, c) not in game.revealed and (r, c) not in self.suspected_mines]
        if not unrevealed:
            return None
        return random.choice(unrevealed)

In [3]:
import os
class Simulation:
    def __init__(self, rows, cols, num_mines, agent, save_dir='simulation_images'):
        """
        Initialize the simulation.

        Parameters:
        - rows (int): Number of rows in the board.
        - cols (int): Number of columns in the board.
        - num_mines (int): Number of mines on the board.
        - agent (Agent): The agent that will play the game.
        - save_dir (str): Directory to save the heatmap images.
        """
        self.game = Minesweeper(rows, cols, num_mines)
        self.agent = agent
        self.save_dir = save_dir
        self.move_count = 0
        self.create_save_directory()

    def create_save_directory(self):
        """
        Create the directory to save images if it doesn't exist.
        """
        if not os.path.exists(self.save_dir):
            os.makedirs(self.save_dir)
        else:
            # Clear existing images
            for filename in os.listdir(self.save_dir):
                file_path = os.path.join(self.save_dir, filename)
                if os.path.isfile(file_path):
                    os.unlink(file_path)

    def run(self):
        """
        Run the simulation until the game is over (win or lose).

        Returns:
        - bool: True if the agent won, False if it hit a mine.
        """
        self.save_mines_image()
        while True:
            # Save current board state
            self.save_board_image()

            # Check for win condition
            if self.check_win():
                print("Agent has successfully cleared the board!")
                return True

            # Agent chooses next move
            move = self.agent.choose_move(self.game)
            if move is None:
                print("No more moves left. Game over.")
                return False

            r, c = move
            print(f"Move {self.move_count + 1}: Revealing cell ({r}, {c})")
            safe = self.game.reveal_square(r, c)
            self.move_count += 1

            if not safe:
                # Save final board state with mine revealed
                self.save_board_image()
                print(f"Agent hit a mine at ({r}, {c}). Game over.")
                return False

    def check_win(self):
        """
        Check if all non-mine cells have been revealed.

        Returns:
        - bool: True if the agent has won, False otherwise.
        """
        total_cells = self.game.rows * self.game.cols
        if len(self.game.revealed) == total_cells - self.game.num_mines:
            return True
        return False
    
    def save_mines_image(self):
        """
        Save the positions of all mines on the board as an image.
        """
        filename = os.path.join(self.save_dir, "mines.png")
        plt.figure(figsize=(10, 8))
        self.game.plot_mines()
        plt.savefig(filename)
        plt.close()

    def save_board_image(self):
        """
        Save the current state of the board as an image.
        """
        filename = os.path.join(self.save_dir, f"move_{self.move_count:04d}.png")
        plt.figure(figsize=(10, 8))       # Create a new figure
        self.game.plot_board()            # Plot on the current figure
        plt.savefig(filename)             # Save the current figure
        plt.close()                       # Close the figure to free memory



In [4]:
rows, cols, num_mines = 10, 10, 15

# Initialize game and AI agent
game = Minesweeper(rows, cols, num_mines)
game.display_board()
# game.plot_board()

 *  2  *  1  0  0  1  2  2  1 
 1  2  2  2  1  0  1  *  *  2 
 0  0  1  *  1  0  1  2  3  * 
 0  0  1  2  2  2  2  2  2  1 
 1  1  1  2  *  3  *  *  1  0 
 2  *  3  3  *  3  2  2  1  0 
 2  *  4  *  2  1  0  0  0  0 
 1  2  *  2  1  0  0  0  0  0 
 0  1  1  1  0  0  0  0  1  1 
 0  0  0  0  0  0  0  0  1  * 



In [11]:
if __name__ == "__main__":
    # Define game parameters
    rows = 10
    cols = 10
    num_mines = 15

    # Initialize agent and simulation
    agent = LogicAgent()
    simulation = Simulation(rows, cols, num_mines, agent, save_dir='simulation_images')

    # Run the simulation
    result = simulation.run()

    if result:
        print("Agent won the game!")
    else:
        print("Agent lost the game.")


Move 1: Revealing cell (3, 4)
Move 2: Revealing cell (1, 2)
Move 3: Revealing cell (3, 7)
Move 4: Revealing cell (1, 1)
Move 5: Revealing cell (7, 0)
Move 6: Revealing cell (5, 7)
Move 7: Revealing cell (1, 0)
Move 8: Revealing cell (4, 8)
Move 9: Revealing cell (6, 6)
Move 10: Revealing cell (4, 7)
Move 11: Revealing cell (7, 8)
Move 12: Revealing cell (2, 8)
Move 13: Revealing cell (0, 1)
Move 14: Revealing cell (3, 8)
Move 15: Revealing cell (9, 0)
Move 16: Revealing cell (0, 0)
Move 17: Revealing cell (6, 8)
Move 18: Revealing cell (0, 9)


/var/folders/bn/llk1pmgn40d3pd2v_zdcks9h0000gn/T/ipykernel_85085/3347290973.py:158: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(10, 10))
/var/folders/bn/llk1pmgn40d3pd2v_zdcks9h0000gn/T/ipykernel_85085/2158832625.py:94: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(10, 8))       # Create a new figure


Move 19: Revealing cell (7, 9)
Move 20: Revealing cell (8, 9)
Agent has successfully cleared the board!
Agent won the game!


<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

...

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>